# Копирование объектов в Python

Если присваивание в Python всегда создает указатель на объект, то как же получить копию объекта?

Изменяемые объекты встроенных классов, например, `list`, `dict`, имеют метод `copy`. Объекты классов, определенных программистом, можно скопировать с помощью функций модуля `copy`.

Но прежде чем продемонстрировать копирование объектов, покажу, когда оно не работает. Копирование не работает для неизменяемых (immutable) объектов, например, для объектов классов `int`, `str`, и других. Метод `copy` у этих классов отсутствует:

In [1]:
(
    hasattr(int, 'copy')
    or hasattr(float, 'copy')
    or hasattr(str, 'copy')
    or hasattr(tuple, 'copy')
    or hasattr(bool, 'copy')
)

False

Функция `copy` модуля `copy`, хотя и не возвращает ошибок при использовании с неизменяемыми объектами, возвращает оригинальный объект, а не копию. Это легко видеть с помощью функции `id()`:

In [2]:
from copy import copy

x = 1               # int
y = copy(x)
print('x', x, id(x))
print('y', y, id(y))

x = 'hi'            # str
y = copy(x)
print('x', x, id(x))
print('y', y, id(y))

x = (0, 1)          # tuple
y = copy(x)
print('x', x, id(x))
print('y', y, id(y))

x 1 1851482336
y 1 1851482336
x hi 1814976807296
y hi 1814976807296
x (0, 1) 1815012745160
y (0, 1) 1815012745160


Почему нельзя скопировать неизменяемый объект? Потому что нет никакого смысла делать это. Ведь состояние такого объекта, его значение, не может измениться, значит, для всех возможных целей достаточно иметь один экземпляр неизменяемого объекта с данным значением. Таким образом оптимизируется использование памяти.

В частности, экземпляры `True` и `False` класса `bool` на протяжении выполнения программы имеют одни и те же идентификаторы, независисмо от способа, которым они получены:

In [3]:
print(id(False), id(True))
print(id(1 != 1), id(1 == 1))
print(id(bool()), id(bool(1)))

1850990848 1850990816
1850990848 1850990816
1850990848 1850990816


Но вернемся к копированию объектов.

Получим копию изменяемого объекта - списка - с помощью метода `copy` и свяжем ее с новым именем.

In [4]:
x = [0, 1]
y = x.copy()
print('x', x, id(x), id(x[0]), id(x[1]))
print('y', y, id(y), id(y[0]), id(y[1]))

x [0, 1] 1815013997576 1851482304 1851482336
y [0, 1] 1815013837960 1851482304 1851482336


Как видим, элементы оригинального списка `x` и списка-копии `y` после копирования указывают на одни и те же объекты, но сами списки - разные. И изменения в них вносятся независимо:

In [5]:
x[0] = 2
y[1] = 3
print('x', x, id(x), id(x[0]), id(x[1]))
print('y', y, id(y), id(y[0]), id(y[1]))

x [2, 1] 1815013997576 1851482368 1851482336
y [0, 3] 1815013837960 1851482304 1851482400


Однако, в случае двумерного списка (и списков б**о**льших размерностей) копирования с помощью метода `copy` недостаточно для получения списка-копии, полностью независимого от оригинала.

Продемонстрирую это:

In [6]:
x = [[0, 0], [1, 1]]
y = x.copy()
print('x', x, id(x), id(x[0]), id(x[1]))
print('y', y, id(y), id(y[0]), id(y[1]))

x [[0, 0], [1, 1]] 1815013997448 1815013997256 1815013997704
y [[0, 0], [1, 1]] 1815013837896 1815013997256 1815013997704


Метод `copy` создал список-копию, содержащий указатели на вложенные списки `[0, 0]` и `[1, 1]`. Но сами вложенные списки не были скопированы, что видно по одинаковым `id` элементов `x[0]` и `y[0]` и элементов `x[1]` и `y[1]`. Следовательно, получить доступ к элементам вложенных списков и изменить их можно как через оригинальный список `x`, так и через список-копию `y`:

In [7]:
x[0][0] = 9
y[1][0] = 9
print('x', x, id(x), id(x[0]), id(x[1]))
print('y', y, id(y), id(y[0]), id(y[1]))

x [[9, 0], [9, 1]] 1815013997448 1815013997256 1815013997704
y [[9, 0], [9, 1]] 1815013837896 1815013997256 1815013997704


Разумеется, вложенные словари демонстрируют такое же поведение. А также словари, значениями которых являются списки, или списки, элементами которых являются словари (или другие изменяемые объекты).

Пока элементы копируемого списка или словаря указывают на неизменяемые объекты, оригинал и копия, полученная с помощью метода `copy`, могут изменяться независимо. Потому что все, что мы можем сделать в таком случае - это присвоить элементу списка или словаря, указывающему на неизменяемый объект, другой объект. Но если элементы копируемого списка или словаря указывают на изменяемые объекты, мы можем получить доступ к последним и изменить их как через копию, так и через оригинал. В этом случае копия и оригинал не являются полностью независимыми.

Так же ведут себя при копировании изменяемые объекты классов, определенных программистом. Пока атрибуты объекта указывают на неизменяемые объекты, значения атрибутов у оригинального объекта и объекта-копии независимы. Воспользуемся функцией `copy` из модуля `copy` для копирования объекта:

In [8]:
from copy import copy

class Cls(object):
    pass

obj = Cls()
obj.greeting = 'привет'
xyz = copy(obj)
print('obj', obj, id(obj), obj.greeting, id(obj.greeting))
print('xyz', xyz, id(xyz), xyz.greeting, id(xyz.greeting))

obj <__main__.Cls object at 0x000001A697438DA0> 1815013985696 привет 1815013944624
xyz <__main__.Cls object at 0x000001A697438F60> 1815013986144 привет 1815013944624


Теперь изменим значение атрибута `greeting` объекта `obj` и убедимся, что атрибут объекта `xyz` остался без изменений:

In [9]:
obj.greeting = 'salut'
print('obj', obj, id(obj), obj.greeting, id(obj.greeting))
print('xyz', xyz, id(xyz), xyz.greeting, id(xyz.greeting))

obj <__main__.Cls object at 0x000001A697438DA0> 1815013985696 salut 1815013848992
xyz <__main__.Cls object at 0x000001A697438F60> 1815013986144 привет 1815013944624


Пусть теперь атрибут объекта указывает на изменяемый объект, например, список. Получим копию объекта:

In [10]:
obj = Cls()
obj.greeting = ['привет', 'salut']
xyz = copy(obj)
print('obj', obj, id(obj), obj.greeting, id(obj.greeting))
print('xyz', xyz, id(xyz), xyz.greeting, id(xyz.greeting))

obj <__main__.Cls object at 0x000001A697456588> 1815014106504 ['привет', 'salut'] 1815013994760
xyz <__main__.Cls object at 0x000001A697438DA0> 1815013985696 ['привет', 'salut'] 1815013994760


Изменяемый объект, список, на который указывает атрибут `greeting`, можно изменять и получать (читать) как через оригинальный объект, так и через объект-копию:

In [11]:
obj.greeting[1] = 'hello'
print('obj', obj, id(obj), obj.greeting, id(obj.greeting))
print('xyz', xyz, id(xyz), xyz.greeting, id(xyz.greeting))

obj <__main__.Cls object at 0x000001A697456588> 1815014106504 ['привет', 'hello'] 1815013994760
xyz <__main__.Cls object at 0x000001A697438DA0> 1815013985696 ['привет', 'hello'] 1815013994760


Функция `copy` из модуля `copy`, как и метод `copy` изменяемых объектов, выполняют *поверхностное копирование*. При этом копируется сам объект с его атрибутами, которые являются указателями на другие объекты. Эти последние объекты не копируются.

Чтобы скопировать объект и все объекты, на которые указывают его атрибуты, а также все объекты, на которые указывают их атрибуты, и так далее, нужно воспользоваться *глубоким копированием*.

В только что рассмотренных примерах заменим поверхностное копирование на глубокое:

In [12]:
from copy import deepcopy

x = [[0, 0], [1, 1]]
y = deepcopy(x)
print('x', x, id(x), id(x[0]), id(x[1]))
print('y', y, id(y), id(y[0]), id(y[1]))

x [[0, 0], [1, 1]] 1815014067080 1815014064776 1815014064968
y [[0, 0], [1, 1]] 1815013996744 1815012523080 1815012562568


Мы видим, что теперь элементы `x[0]` и `y[0]`, а также элементы `x[1]` и `y[1]`, указывают на разные объекты. Значит, эти объекты - вложенные списки - теперь изменяются независимо:

In [13]:
x[0][0] = 9
y[1][0] = 9
print('x', x, id(x), id(x[0]), id(x[1]))
print('y', y, id(y), id(y[0]), id(y[1]))

x [[9, 0], [1, 1]] 1815014067080 1815014064776 1815014064968
y [[0, 0], [9, 1]] 1815013996744 1815012523080 1815012562568


Пример с глубоким копированием объекта с атрибутом - списком:

In [14]:
obj = Cls()
obj.greeting = ['привет', 'salut']
xyz = deepcopy(obj)
print('obj', obj, id(obj), obj.greeting, id(obj.greeting))
print('xyz', xyz, id(xyz), xyz.greeting, id(xyz.greeting))

obj <__main__.Cls object at 0x000001A697456940> 1815014107456 ['привет', 'salut'] 1815014067784
xyz <__main__.Cls object at 0x000001A697456E10> 1815014108688 ['привет', 'salut'] 1815014067336


Теперь списки, на которые указывает атрибут `greeting` у объекта-оригинала и объекта-копии, - разные (так как имеют разные `id`). Следовательно, они изменяются независимо: 

In [15]:
obj.greeting[1] = 'hello'
print('obj', obj, id(obj), obj.greeting, id(obj.greeting))
print('xyz', xyz, id(xyz), xyz.greeting, id(xyz.greeting))

obj <__main__.Cls object at 0x000001A697456940> 1815014107456 ['привет', 'hello'] 1815014067784
xyz <__main__.Cls object at 0x000001A697456E10> 1815014108688 ['привет', 'salut'] 1815014067336


Подведем итог:
- в Python возможно копирование только изменяемых объектов, а неизменяемые объекты скопировать нельзя,
- копирование объектов с помощью метода `copy` или функции `copy` модуля `copy` является поверхностным в том смысле, что атрибуты объекта-копии указывают на те же самые объекты, на которые указывают атрибуты объекта-оригинала,
- для копирования объекта и рекурсивно всех объектов, на которые он ссылается, существует глубокое копирование, реализуемое функцией `deepcopy` модуля `copy`.